### import packages

In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import random


### Custom Dataset Class for Loading Images from Folders in PyTorch

In [1]:
class ImageFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Filter out non-directory files (like .DS_Store)
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                if os.path.isfile(img_path):  # Make sure it's a file, not a directory
                    images.append((img_path, self.class_to_idx[cls]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label



### Data Augmentation, Normalization, and Dataloading for Image Classification in PyTorch

In [2]:
# Data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Randomly resize and crop the image to 224x224
        transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  # Resize the image to 256x256
        transforms.CenterCrop(224),  # Crop the center 224x224 portion
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the correct path to your dataset
data_dir = '/content/drive/My Drive/NWPU-RESISC45-classification/'  # Root dataset directory

# Load datasets - point directly to the main dataset directory (containing class folders)
image_datasets = {
    'train': ImageFolderDataset(data_dir, data_transforms['train']),
    'val': ImageFolderDataset(data_dir, data_transforms['val'])  # Assuming validation data is the same structure
}

# Create dataloaders to feed the data into the model
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4)
}

# Get dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Use GPU if available


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### Creating a Fine-Tuned ResNet50 Model with Custom Fully Connected Layer

In [3]:
# Function to create a ResNet50 model for fine-tuning
def create_model(num_classes):
    # Load the pre-trained ResNet50 model
    model = models.resnet50(pretrained=True)

    # Freeze the earlier layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace the fully connected layer to match the number of classes in your dataset
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)  # Modify the last layer

    return model


### Training and Validation Function for Fine-Tuning ResNet50 in PyTorch

In [4]:
# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # Backward pass + optimization in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model with the best validation accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    print(f'Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)  # Load the best model weights
    return model


## Main Function to Fine-Tune ResNet50 and Save the Best Model in PyTorch

In [5]:
# Main function to run the experiment
def run_experiment():
    num_classes = len(class_names)  # Get the number of classes for classification
    model = create_model(num_classes).to(device)  # Create the ResNet50 model and move it to GPU or CPU
    criterion = nn.CrossEntropyLoss()  # Define the loss function (CrossEntropy for classification)

    # Define optimizer and learning rate scheduler
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # Train the model using the train_model function
    model = train_model(model, criterion, optimizer, scheduler, num_epochs=7)

    # Save the fine-tuned model
    torch.save(model.state_dict(), 'resnet50_finetuned_best.pth')

# Run the experiment when the script is executed
if __name__ == "__main__":
    run_experiment()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 181MB/s]


Epoch 0/4
----------
train Loss: 2.0853 Acc: 0.5574
val Loss: 1.1071 Acc: 0.7517

Epoch 1/4
----------
train Loss: 1.2434 Acc: 0.6948
val Loss: 0.8644 Acc: 0.7766

Epoch 2/4
----------
train Loss: 1.0773 Acc: 0.7200
val Loss: 0.7601 Acc: 0.7928

Epoch 3/4
----------
train Loss: 0.9985 Acc: 0.7316
val Loss: 0.7091 Acc: 0.8024

Epoch 4/4
----------
train Loss: 0.9467 Acc: 0.7412
val Loss: 0.6727 Acc: 0.8087

Best val Acc: 0.8087



| Epoch | Train Loss | Train Accuracy | Val Loss | Val Accuracy |
|-------|------------|----------------|----------|--------------|
| 0     | 2.0853     | 0.5574         | 1.1071   | 0.7517       |
| 1     | 1.2434     | 0.6948         | 0.8644   | 0.7766       |
| 2     | 1.0773     | 0.7200         | 0.7601   | 0.7928       |
| 3     | 0.9985     | 0.7316         | 0.7091   | 0.8024       |
| 4     | 0.9467     | 0.7412         | 0.6727   | 0.8087       |
